## 1. src 文件夹：源模型 (Source Models)      
目的：src 文件夹通常用于存放源模型（Source Models），即直接从数据源（例如原始表、外部 API 数据等）导入的数据。这些模型通常不进行太多的数据转换，而是简单地引用原始数据表，并根据需要进行必要的数据清理。

特点：

这些模型通常是 "原始" 或 "未经处理" 的数据。
它们主要用来创建数据库中的表（或视图），以便后续的转换工作可以在其上进行。
在 DBT 中，这些模型可能会通过 source() 函数来引用。例如，src_job_ads 可能是从名为 job_ads 的数据库表或视图中提取的数据。
示例：

src_job_ads.sql：这个模型可能从一个叫 job_ads 的源表中读取数据，并进行轻微的清理或标准化操作。
sql
Kopiera
Redigera
-- src_job_ads.sql
select * from {{ source('job_ads', 'stg_data_ads') }}

## 2. dim 文件夹：维度模型 (Dimension Models)
目的：dim 文件夹通常用于存放维度模型。维度模型通常是经过转换的结构化数据，通常是为了在数据仓库中进行分析而设计的。这些模型通常通过 数据清洗、转换、标准化 等步骤，从原始数据中提取出有用的信息。

特点：

维度模型通常包含了 描述性信息，比如时间维度、产品维度、用户维度等。
它们会经过一些数据处理，例如去重、填充缺失值、标准化字段名称等。
维度模型常常用来和事实模型（fact model）结合，用于分析数据。
示例：

dim_employer.sql：这个模型可能会从源数据表（如 src_job_ads）中提取字段，并根据业务需求进行清理和转换，形成一个规范化的维度表。

## 3. fct 文件夹：事实模型 (Fact Models)
目的：fct 文件夹通常用于存放事实表（Fact Tables）。事实表包含了量化的数据（如销售金额、访问次数、交易数量等），并且通常包含多个外键来链接到相应的维度表。

特点：

事实表通常包含 度量数据，例如销售额、交易数量、收入等。
它们经常需要与 dim 文件夹中的维度表进行连接，以便进行更复杂的分析。
这些模型经常包含对多个数据源的聚合或计算，以便得到最终的分析结果。
示例：

fct_sales.sql：这个模型可能会汇总销售数据，并计算总销售额、订单数量等。
sql
Kopiera
Redigera
-- fct_sales.sql
select
    order_id,
    customer_id,
    sum(order_amount) as total_sales,
    count(*) as total_orders,
    ...
from {{ ref('src_sales') }}
group by order_id, customer_id

## 4. stg 文件夹：中间层模型 (Staging Models)
目的：stg 文件夹通常用于存放中间层模型，即对源数据进行初步清理、转换或格式化的模型。它们并不直接参与业务分析，而是将源数据转换成一个更标准化的格式，供后续的模型（例如 dim 或 fct）使用。

特点：

中间层模型通常会进行数据清洗，例如去重、数据类型转换、列重命名等。
它们主要是为了提高数据的质量和一致性，以便后续模型使用。
这些模型一般与源数据表进行直接引用，通常会通过 source() 函数来访问原始数据。
示例：

stg_sales.sql：将原始的销售数据清理、去重、标准化后存储。
sql
Kopiera
Redigera
-- stg_sales.sql
select
    order_id,
    customer_id,
    order_amount,
    order_date,
    ...
from {{ source('sales_data', 'raw_sales') }}


## 5. marts 文件夹：数据集市模型 (Data Mart Models)
目的：marts 文件夹通常用于存放数据集市（Data Mart）模型，数据集市是数据仓库的子集，专门为特定的业务部门或团队（如销售、市场、财务等）提供分析所需的数据。这些模型通常是从 dim 和 fct 模型中派生的，以便支持更具体的业务需求。

特点：

数据集市模型通常是基于业务需求进行设计的，目的是为特定的业务线提供数据。
它们通常包括 聚合数据 或 特定维度的分析。
示例：

marts_sales.sql：为销售团队构建的模型，可能聚合了不同地区或产品线的销售数据。


## source() 和 ref()：

source() 用于引用外部数据源（通常是在 src 文件夹中定义的），例如外部数据库的表或视图。
ref() 用于引用其他 DBT 模型，通常用于引用其他模型文件中的转换结果。

models/
  ├── src/
  │    └── src_job_ads.sql    # 定义了外部数据源
  ├── stg/
  │    └── stg_job_ads.sql    # 对 job_ads 进行清洗
  ├── dim/
  │    └── dim_job_ads.sql    # 定义 job_ads 的维度模型
  └── fct/
       └── fct_job_ads.sql    # 创建基于 job_ads 的事实表



总结顺序
首先，执行 stg 模型进行数据清理和预处理。
然后，执行 dim 模型来创建维度表。
接着，执行 fct 模型来创建事实表。
再后，执行 marts 模型来创建数据集市模型。
然后，可以运行 snapshots 模型来跟踪数据的历史变化。
最后，运行 test 模型来验证数据质量。

1. src：存放源数据模型，通常是从原始数据表中读取的模型。
2. dim：存放维度模型，通常用于描述性数据，例如用户、产品等。
3. fct：存放事实模型，通常包含量化的度量数据，如销售额、交易次数等。
4. stg：存放中间层模型，主要用于清理和转换源数据，为后续的模型做准备。
5. marts：存放数据集市模型，通常面向特定业务领域的分析。
6. test：存放数据质量测试模型，用于验证数据的准确性。


简化 DBT 学习的几个要点
模型文件（models）：这就是你定义 SQL 查询的地方，处理从数据源到目标模型的转换。这里是最核心的部分，你定义了业务逻辑、数据清洗、汇总等内容。

文件夹结构：

- models 文件夹：通常包含各类模型文件，你可以按照业务需求分成多个子文件夹，像 staging（暂存层）、dim（维度层）、fct（事实表层）等，主要是帮助你更好地组织模型和数据流。
- macros 文件夹：存放重用的 SQL 代码片段，也就是定义一些函数或逻辑，方便在多个模型中调用。
- seeds 文件夹：用来存储小规模的数据文件（例如 CSV 文件），这些数据文件在 DBT 项目中可以作为输入来运行模型。
- snapshots 文件夹：用来跟踪表的变化，保存历史数据版本，适用于需要保持历史记录的场景。
- 配置文件（dbt_project.yml 和 profiles.yml）：

dbt_project.yml 配置文件定义了你的 DBT 项目的结构，包括模型文件路径、数据源、目标数据库等信息。
profiles.yml 文件配置了你与目标数据库（如 Snowflake, BigQuery 等）的连接信息。

- DBT 运行：
通过 dbt run 命令，你可以自动化整个数据建模过程。它会根据你定义的模型、依赖关系和数据库连接，自动执行 SQL 查询，并将结果保存到目标数据库。
你可以先从一个简单的模型文件入手，逐步理解 ref()、source() 等概念。

核心概念：

模型（Model）：你的数据变换的核心 SQL 文件。
数据源（Source）：外部数据源的引用。
引用（Ref）：模型间的依赖管理。

理解数据流：DBT 的强大之处在于能够通过 ref() 自动处理模型间的依赖关系。你可以从最简单的模型入手，逐渐了解如何定义和调度复杂的数据流。
探索 Macros 和 Snapshots：逐渐理解如何利用宏（macros）简化复杂 SQL 查询，并且可以用 snapshots 进行数据版本控制。
使用 DBT 的文档和社区资源：


ref() 是 DBT 中的一个宏，它帮助你管理模型之间的依赖关系，保证模型按正确顺序执行。

代理键（Surrogate Key）是什么？
代理键是一个人为生成的唯一标识符，通常用于替代真实的业务主键。代理键一般是自动生成的，通常是递增的数字（例如：1, 2, 3...），目的是保持数据表中记录的唯一性，尤其是在数据仓库中，不依赖于业务逻辑中的主键（如 id、name 等）。

为什么要使用 generate_surrogate_key？
在数据仓库的建模中，尤其是当我们合并不同来源的数据时，我们通常希望有一个 统一的唯一标识符 来标识一条记录。generate_surrogate_key 这个宏可以根据传入的字段自动生成一个 哈希值，从而保证每条记录有一个唯一的 ID。

generate_surrogate_key 怎么工作？
generate_surrogate_key 是一个来自 dbt_utils 的宏，它会根据你提供的字段列表生成一个唯一的哈希值。哈希值的生成通常会对输入的字段（例如：id 和 headline）进行处理，以确保每一组数据组合有一个唯一的标识符。具体过程如下：

你传入一个字段数组：['id', 'headline']。
generate_surrogate_key 会对这些字段的值进行哈希处理（类似于生成一个签名）。
结果会是一个唯一的值，可以用作数据库中的代理键。
示例：
假设你有以下数据表（表名：job_details）：

id	headline	description
1	"Software Dev"	"Job in tech"
2	"Data Analyst"	"Job with data"
1	"Software Dev"	"Job in tech"
如果你直接使用 id 作为主键，第二行和第三行的记录会冲突，因为它们的 id 都是 1。这时，你可以用 id 和 headline 这两个字段的组合来生成一个唯一的代理键。
代理键生成过程：
使用 generate_surrogate_key(['id', 'headline']) 宏，你会得到这样的结果：

第一行的 job_details_id 可能是一个哈希值，例如：'f56b77fd8d2c91a62b9a2128'。
第二行的 job_details_id 可能是：'ba2d37a58b0c29d35d20e240'。
第三行的 job_details_id 和第一行相同，因为 id 和 headline 这两个字段的组合相同。
通过这种方式，即使原始数据的 id 相同，生成的 job_details_id 也会不同，确保每条记录都可以唯一标识。

例子：如何使用
假设有以下表格：

id	headline
1	"Software Dev"
2	"Data Analyst"
1	"Software Dev"
使用 generate_surrogate_key(['id', 'headline']) 宏生成的代理键会像下面这样：

id	headline	job_details_id
1	"Software Dev"	f56b77fd8d2c91a62b9a2128
2	"Data Analyst"	ba2d37a58b0c29d35d20e240
1	"Software Dev"	f56b77fd8d2c91a62b9a2128
这个 job_details_id 就是一个代理键，它通过组合了 id 和 headline 字段来生成一个唯一的标识符。

总结：
{{ dbt_utils.generate_surrogate_key(['id', 'headline']) }} AS job_details_id 这段代码的目的是：

通过组合 id 和 headline 字段，生成一个 唯一的哈希值，作为每条记录的代理键 job_details_id。
代理键 job_details_id 会在 src_job_details 表的基础上创建，确保每条记录都能唯一标识，避免使用可能会重复的业务主键（如 id）。

在 dbt 中，事实表的主键（Primary Key, PK）通常是在 SQL 查询中创建的，并没有像传统数据库那样显式定义主键约束（因为 dbt 主要是在数据仓库中生成表或视图，而不是管理约束）
总结
📌 dbt 不直接定义 PRIMARY KEY，但可以通过：

generate_surrogate_key() 生成合成键
在 schema.yml 里用 tests 确保唯一性
在数据仓库里创建索引（如果需要）

## DBT Documentation

DBT Documentation（DBT 文档）是 DBT（Data Build Tool） 提供的一种自动化数据模型文档生成方式。     
DBT 可以帮助数据团队 构建、测试、文档化 数据转换流程，确保数据可追踪、可理解。           

- DBT Documentation 作用       
✅ 自动化数据文档：DBT 通过 YAML 配置文件 生成文档，无需手动写文档。       
✅ 可视化数据血缘：可以 查看数据来源（Lineage），了解字段如何在不同模型中流转。        
✅ 生成网页文档：DBT 提供 dbt docs serve 命令，可以本地启动 Web 界面，直观查看数据表结构、字段描述等。        
✅ 增强数据治理：让数据工程师、分析师更容易理解数据模型，提高协作效率       

- 如何使用 DBT Documentation？      
1️⃣ 添加文档注释 在 dbt models 的 YAML 文件 中 schema.yml ，添加 描述字段：     
2️⃣ 生成文档 运行:  这将扫描你的 models/ 目录，生成 HTML 文档（通常是 target/catalog.json）      
bash     
dbt docs generate      
"Catalog written to /Users/apple/Data_Engineer_Tools/Data_Warehouse/github/Data_Warehouse_JC/code_alongs/14_dbt_documentation/dbt_code/target/catalog.json"      

3️⃣ 本地启动 Web 界面             
bash          
dbt docs serve           
然后打开浏览器，访问 **http://localhost:8080**，就能看到 数据血缘关系 和 字段描述。            

- 📌 DBT Documentation 生成的内容         
📄 表（Model）信息：显示数据表的 名称、描述                   
📌 字段（Column）描述：每个字段的 含义 和 数据类型            
🔗 数据血缘（Lineage）：可视化展示 数据如何从一个表流向另一个表          
✅ 测试（Tests）结果：显示哪些数据转换规则被验证通过           

- 📌 总结            
DBT Documentation 自动化生成数据文档，结合 Web UI，让数据分析师、工程师更直观地理解数据模型、追踪数据血缘，提升数据治理能力！          